## Importing query files

In [ ]:
import pandas as pd

target_og_query_file_path = ""
target_rephrased_query_file_path = ""
control_query_file_path = ""

target_og_queries_df = pd.read_excel(target_og_query_file_path)
target_rephrased_queries_df = pd.read_excel(target_rephrased_query_file_path)
control_queries_df = pd.read_excel(control_query_file_path)

## Bing

In [ ]:
import json
from tqdm.notebook import tqdm
import httpx
from urllib.request import Request, urlopen
from inscriptis import get_text
from datetime import datetime

f = open("API_keys.json")
data = json.load(f)

key = data["bing"]["key"]
SERP_endpoint = data["bing"]["SERP_endpoint"]
location = data["bing"]["location"]

f.close()

# headers = {
#             'Ocp-Apim-Subscription-Key': key1,
            
#         }

headers = {
            'Ocp-Apim-Subscription-Key': key,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'en-US,en;q=0.9',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
        }

def get_bing_results(queries_df, result_file_name):
    SERP_results = []
    today = datetime.now()
    today = today.strftime("%Y_%m_%d")

    # queries_df = queries_df.iloc[range(0,5)]

    for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
        query = row["query"] 
        query_category = str(row["ngram"]) + '-gram'

        params = {
            'q': query,
            'answerCount': 10, # number of results to be displayed
            'setLang': 'en',
            'mkt': 'en-US' # english in US (local market resulting in results in Dutch)
        }

        SERP_response = httpx.get(url=SERP_endpoint, headers=headers, params=params)
        try:
            SERP_result_set = SERP_response.json()
            asked_query = SERP_result_set['queryContext']['originalQuery']
            for result in SERP_result_set['webPages']['value']:
                web_title =  result["name"]
                web_url = result["url"]
                web_snippet = result["snippet"]
                # try:
                #     req = Request(
                #         url= web_url, 
                #         headers={'User-Agent': 'Mozilla/5.0'}
                #     )
                #     html = urlopen(req).read().decode('utf-8')
                #     # html = urllib.request.urlopen(web_url).read().decode('utf-8')
                #     # web_text = get_text(html) 
                # except Exception as error:
                #     # print(error)
                #     # print(web_url)
                #     web_text = None
                
                # SERP_results.append([query, query_category, web_title, web_url, web_snippet, web_text, today])
                SERP_results.append([query, asked_query, query_category, web_title, web_url, web_snippet, today])
        except Exception as error:
            print(error)
            # print(reformed_query)
            SERP_results.append([query, asked_query, query_category, None, None, None, today])

    # SERP_df = pd.DataFrame(SERP_results, columns=["query", "query_category", "web_title", "web_url", "web_snippet", "web_text", "date_crawled"])
    SERP_df = pd.DataFrame(SERP_results, columns=["query", "asked_query", "query_category", "web_title", "web_url", "web_snippet", "date_crawled"])
    SERP_df.to_excel("../data/" + result_file_name + ".xlsx", index=False)


  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
get_bing_results(queries_df=target_og_queries_df, result_file_name="Bing_results")
get_bing_results(queries_df=target_rephrased_queries_df, result_file_name="Bing_QueryReformed_results")
get_bing_results(queries_df=control_queries_df, result_file_name="Control_Bing_results")

## Google

In [ ]:
from datetime import datetime 
import requests
import pandas as pd
import json
from tqdm.notebook import tqdm

f = open("API_keys.json")
data = json.load(f)

API_KEY = data["google"]["api_key"]
SEARCH_ENGINE_ID = data["google"]["search_engine_id"]
SERP_endpoint = data["google"]["SERP_endpoint"]
f.close()

# payload = {
#         'key': API_KEY,
#         'q': query,
#         'cx': SEARCH_ENGINE_ID
#     }

# SERP_response = requests.get(url=SERP_endpoint, params=payload)
# SERP_result_set = SERP_response.json()
# SERP_result_set["items"]

def get_google_results(queries_df, results_file_name):

    SERP_results = []
    today = datetime.now()
    today = today.strftime("%Y_%m_%d")

    for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
        query = row["query"] 
        query_category = str(row["ngram"]) + '-gram'

        payload = {
            'key': API_KEY,
            'q': query,
            'cx': SEARCH_ENGINE_ID,
            'lr': "lang_en"
        }

        SERP_response = requests.get(url=SERP_endpoint, params=payload)
        try:
            SERP_result_set = SERP_response.json()
            asked_query = payload['q']
            for result in SERP_result_set["items"]:
                try:
                    web_title =  result["title"]
                except:
                    web_title = None
                try:
                    web_url = result["link"]
                except:
                    web_url = None
                try:
                    web_snippet = result["snippet"]
                except:
                    web_snippet = None

                # try:
                #     req = Request(
                #         url= web_url, 
                #         headers={'User-Agent': 'Mozilla/5.0'}
                #     )
                #     html = urlopen(req).read().decode('utf-8')
                #     # html = urllib.request.urlopen(web_url).read().decode('utf-8')
                #     # web_text = get_text(html) 
                # except Exception as error:
                #     # print(error)
                #     # print(web_url)
                #     web_text = None
                
                SERP_results.append([query, asked_query, query_category, web_title, web_url, web_snippet, today])
        except:
            print(query)
            SERP_results.append([query, asked_query, query_category, None, None, None, today])

    SERP_df = pd.DataFrame(SERP_results, columns=["query", "asked_query", "query_category", "web_title", "web_url", "web_snippet", "date_crawled"])
    SERP_df.to_excel("../data/" + results_file_name + ".xlsx", index=False)

  0%|          | 0/250 [00:00<?, ?it/s]

MISSDAWBB.NET
"www.xltits.net"


In [ ]:
get_google_results(queries_df=target_og_queries_df, result_file_name="Google_results")
get_google_results(queries_df=target_rephrased_queries_df, result_file_name="Google_QueryReformed_results")
get_google_results(queries_df=control_queries_df, result_file_name="Control_Google_results")

## Gemini

In [ ]:
import pandas as pd
from vertexai.preview.generative_models import GenerativeModel
from datetime import datetime 
from tqdm.notebook import tqdm

model = GenerativeModel("gemini-pro-vision")

In [ ]:
def get_gemini_results(queries_df, results_file_name):

    SERP_results = []
    # queries_df = queries_df.iloc[range(0,5)]
    today = datetime.now()
    today = today.strftime("%Y_%m_%d")
    failed_query = 0

    for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
        query = row["query"]
        query_category = str(row["ngram"]) + '-gram'

        response = model.generate_content(
            [query]
        )

        try:
            response_text = response.candidates[0].content.parts[0].text
        except:
            failed_query += 1

        SERP_results.append([query, query_category, response_text, today])

    SERP_df = pd.DataFrame(SERP_results, columns=["query", "query_category", "response_text", "date_crawled"])
    SERP_df.to_excel("../data/" + results_file_name + ".xlsx", index=False)
    print("No repsonse generated for ", failed_query, " queries.")

  0%|          | 0/250 [00:00<?, ?it/s]

No repsonse generated for  2  queries.


In [ ]:
get_gemini_results(queries_df=target_og_queries_df, result_file_name="Gemini_results")
get_gemini_results(queries_df=target_rephrased_queries_df, result_file_name="Gemini_QueryReformed_results")
get_gemini_results(queries_df=control_queries_df, result_file_name="Control_Gemini_results")

## ChatGPT

In [ ]:
from openai import OpenAI
import json
from datetime import datetime 
from tqdm.notebook import tqdm

f = open("API_keys.json")
data = json.load(f)

API_KEY = data["chatgpt"]["api_key"]
f.close()

client = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
)

model_id = 'gpt-3.5-turbo'

def get_gpt_results(queries_df, results_file_name):

    SERP_results = []
    # queries_df = queries_df.iloc[range(0,5)]
    today = datetime.now()
    today = today.strftime("%Y_%m_%d")
    failed_query = 0

    for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
        query = row["query"]
        # reformed_query = "I am autistic please explain " +  query + " to me"
        query_category = str(row["ngram"]) + '-gram'

        response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
            model=model_id,
        )

        try:
            response_text = response.choices[0].message.content
        except:
            failed_query += 1
        SERP_results.append([query, query_category, response_text, today])

    SERP_df = pd.DataFrame(SERP_results, columns=["query", "query_category", "response_text", "date_crawled"])
    SERP_df.to_excel("../data/" + results_file_name + ".xlsx", index=False)
    print("No repsonse generated for ", failed_query, " queries.")

  0%|          | 0/250 [00:00<?, ?it/s]

No repsonse generated for  0  queries.


In [ ]:
get_gpt_results(queries_df=target_og_queries_df, result_file_name="ChatGPT_results")
get_gpt_results(queries_df=target_rephrased_queries_df, result_file_name="ChatGPT_QueryReformed_results")
get_gpt_results(queries_df=control_queries_df, result_file_name="Control_ChatGPT_results")